# Imports

In [465]:
from numpy.random import seed
seed(888)
import tensorflow as tf
#import tensorflow.compat.v1 as tf1
#tf1.disable_v2_behavior()
tf.random.set_seed(404)

In [466]:
import os
import numpy as np

from time import strftime
from PIL import Image

# Constants

In [467]:
X_TRAIN_PATH = "MNIST/digit_xtrain.csv"
X_TEST_PATH = "MNIST/digit_xtest.csv"
Y_TRAIN_PATH = "MNIST/digit_ytrain.csv"
Y_TEST_PATH = "MNIST/digit_ytest.csv"

LOGGING_PATH = 'tensorboard_mnist_digit_logs/'

NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_HEIGHT*IMAGE_WIDTH

# Data

In [468]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)

Wall time: 1.52 s


In [469]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

In [470]:
%%time
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)

Wall time: 53.4 s


In [471]:
%%time
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

Wall time: 8.57 s


## EDA

In [472]:
x_train_all.shape

(60000, 784)

In [473]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [474]:
x_test.shape

(10000, 784)

In [475]:
y_train_all[:5]

array([5, 0, 4, 1, 9])

## Data Pre-processing

In [476]:
# Re-scale
x_train_all, x_test = x_train_all / 255.0, x_test / 255.0

In [477]:
# One Hot Encoding of target values
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [478]:
y_train_all.shape

(60000, 10)

In [479]:
y_test = np.eye(NR_CLASSES)[y_test]
y_test.shape

(10000, 10)

### Validation dataset from training data

In [480]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]

x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [481]:
x_train.shape

(50000, 784)

In [482]:
x_val.shape

(10000, 784)

## Setup Tensorflow Graph

In [516]:
X = tf.compat.v1.placeholder(tf.float32, shape=[None, TOTAL_INPUTS], name='X')
Y = tf.compat.v1.placeholder(tf.float32, shape=[None, NR_CLASSES], name='labels')

### Neural Network Architecture

Hyperparameters

In [517]:
nr_epochs = 50
learning_rate = 1e-3

n_hidden1 = 512
n_hidden2 = 64

In [518]:
def setup_layer(input, weight_dim, bias_dim, name):
    with tf.name_scope(name):
        inital_w = tf.random.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        w = tf.Variable(initial_value=inital_w, name='W')

        iniital_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=iniital_b, name='B')

        layer_in = tf.matmul(input, w) + b
        
        if name=='out':
            layer_out = tf.nn.softmax(layer_in)
        else:
            layer_out = tf.nn.relu(layer_in)
        
        tf.compat.v1.summary.histogram('weights', w)
        tf.compat.v1.summary.histogram('biases', b)
        
        return layer_out

In [519]:
layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1],
                      bias_dim=[n_hidden1], name='layer_1')

layer_drop = tf.nn.dropout(layer_1, rate=0.2, name='dropout_layer')

layer_2 = setup_layer(layer_drop, weight_dim=[n_hidden1, n_hidden2],
                      bias_dim=[n_hidden2], name='layer_2')

output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES],
                      bias_dim=[NR_CLASSES], name='out')

model_name = f'{n_hidden1}-{n_hidden2}-DO-LR{learning_rate} E{nr_epochs}'

## Tensorboard Setup

In [520]:
# Folder Setup

folder_name = f'{model_name} at {strftime("%H-%M")}'
directory = os.path.join(LOGGING_PATH, folder_name)
try:
    os.makedirs(directory)
except OSError as exception:
    print(exception)
else:
    print('Successfully created directories!')

Successfully created directories!


## Loss, Optimization & Metrices

In [521]:
with tf.name_scope('loss_calc'):
    loss = tf.reduce_mean(tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output))

In [522]:
with tf.name_scope('optimizer'):
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
    train_step = optimizer.minimize(loss)

In [523]:
with tf.name_scope('accuracy_calc'):
    model_preds = tf.argmax(output, axis=1, name='prediction')
    correct_pred = tf.equal(tf.argmax(output, axis=1), tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [524]:
with tf.name_scope('performance'):
    tf.compat.v1.summary.scalar('accuracy', accuracy)
    tf.compat.v1.summary.scalar('cost', loss)

#### Check input Images in Tensorboard

In [525]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X, [-1, 28, 28, 1])
    tf.compat.v1.summary.image('image_input', x_image, max_outputs=4)

### Session

In [526]:
sess = tf.compat.v1.Session()

#### Setup Filewriter and Merge Summaries

In [527]:
merged_summary = tf.compat.v1.summary.merge_all()

train_writer = tf.compat.v1.summary.FileWriter(directory + '/train')
train_writer.add_graph(sess.graph)

validation_writer = tf.compat.v1.summary.FileWriter(directory + '/validation')

#### initialize the variables

In [528]:
init = tf.compat.v1.global_variables_initializer()
sess.run(init)

### Batching Data

In [529]:
size_of_batch = 1000
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0

In [530]:
def next_batch(batch_size, data, labels):
    
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch  += batch_size
    
    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
    
    end = index_in_epoch
    
    return data[start:end], labels[start:end]

### Training loop

In [531]:
for epoch in range(nr_epochs):
    
    #============================Training Data============================
    for i in range(nr_iterations):
        
        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)
        
        feed_dictionary = {X:batch_x, Y:batch_y}
        
        sess.run(train_step, feed_dict=feed_dictionary)
        
    s, batch_accuracy = sess.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)
    
    train_writer.add_summary(s, epoch)
    
    print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')
    
    #==========================Validation Data===============================
    
    summary = sess.run(fetches=merged_summary, feed_dict={X:x_val, Y:y_val})
    validation_writer.add_summary(summary, epoch)

print('Done Training')

Epoch 0 	| Training Accuracy = 0.8539999723434448
Epoch 1 	| Training Accuracy = 0.8640000224113464
Epoch 2 	| Training Accuracy = 0.8640000224113464
Epoch 3 	| Training Accuracy = 0.8700000047683716
Epoch 4 	| Training Accuracy = 0.8740000128746033
Epoch 5 	| Training Accuracy = 0.8730000257492065
Epoch 6 	| Training Accuracy = 0.9710000157356262
Epoch 7 	| Training Accuracy = 0.9819999933242798
Epoch 8 	| Training Accuracy = 0.9800000190734863
Epoch 9 	| Training Accuracy = 0.984000027179718
Epoch 10 	| Training Accuracy = 0.9829999804496765
Epoch 11 	| Training Accuracy = 0.9810000061988831
Epoch 12 	| Training Accuracy = 0.9860000014305115
Epoch 13 	| Training Accuracy = 0.9869999885559082
Epoch 14 	| Training Accuracy = 0.9860000014305115
Epoch 15 	| Training Accuracy = 0.9890000224113464
Epoch 16 	| Training Accuracy = 0.9900000095367432
Epoch 17 	| Training Accuracy = 0.9869999885559082
Epoch 18 	| Training Accuracy = 0.9909999966621399
Epoch 19 	| Training Accuracy = 0.99099999

## Saving the Model

In [532]:
outputs = {'accuracy_calc/prediction': model_preds}
inputs = {'X':X}
tf.compat.v1.saved_model.simple_save(sess,'SavedModel', inputs, outputs)

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: SavedModel\saved_model.pb


## Make Prediction

In [501]:
img = Image.open('MNIST/test_img.png')
img

In [502]:
bw = img.convert('L')
img_array = np.invert(bw)

In [503]:
img_array.shape

(28, 28)

In [504]:
test_img = img_array.ravel()
test_img.shape

(784,)

In [505]:
pred = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X: [test_img]})

In [506]:
print(f'Prediction for test image is {pred}')

Prediction for test image is [2]


## Testing and Evaluation

In [533]:
test_accuracy = sess.run(fetches=accuracy, feed_dict={X:x_test, Y:y_test})
print(f'Accuracy on test set is {test_accuracy:0.2%}')

Accuracy on test set is 97.92%


## Reset for next run

In [534]:
train_writer.close()
validation_writer.close()
sess.close()
tf.compat.v1.reset_default_graph()